In [1]:
import glob
import time
import numpy as np
import pandas as pd
from geopy import distance
import datetime as datetime
from multiprocessing import Process
from sklearn.preprocessing import MinMaxScaler



In [2]:
def create_datetime_features(input_df, column_name):

    input_df[column_name+'_hour'] = input_df[column_name].dt.hour
    input_df[column_name+'_minute'] = input_df[column_name].dt.minute
    input_df[column_name+'_quarter'] = input_df[column_name].dt.quarter
    input_df[column_name+'_month'] = input_df[column_name].dt.month
    input_df[column_name+'_year'] = input_df[column_name].dt.year
    input_df[column_name+'_week'] = input_df[column_name].dt.isocalendar().week
    input_df[column_name+'_day'] = input_df[column_name].dt.day
    input_df[column_name+'_dayofweek'] = input_df[column_name].dt.dayofweek

    return input_df

In [3]:
def calculate_geodesic_distance(df_stations, point):
    df = df_stations.copy()
    for idx, i, j  in zip(df.index, df.station_latitude, df.station_longitude):
        df.loc[idx, 'distance'] = distance.distance(point, (i,j)).m
        if int(df.loc[idx, 'distance']) == 0:
            
            return df.loc[idx, "station_name"], df.loc[idx, 'distance']
    df = df.sort_values("distance")
    return df.iloc[0]["station_name"], df.iloc[0]["distance"]

In [4]:
def get_approximate_stations_locations(df_missing_stations, start_stations, save_file, col ="start_station"):
    count = 0
    for idx, i, j in zip(df_missing_stations.index, 
                         df_missing_stations[f'{col}_latitude'], 
                         df_missing_stations[f'{col}_longitude']):
        
        approx_station, approx_distance = calculate_geodesic_distance(start_stations, 
                            (i, j))
        df_missing_stations.loc[idx,f"{col}_approx"] = approx_station
        df_missing_stations.loc[idx,f"{col}_approx_distance"] = approx_distance
        
        if count%1000 == 0:
            print(count)
            df_missing_stations.to_csv(f"{save_file}")
        count += 1
    df_missing_stations.to_csv(f"{save_file}")
    return df_missing_stations

In [5]:
def parallel_get_approximate_stations_locations(df_missing_stations, start_stations, 
                            save_file_suffix="approx_stations", batch_size=100000, sleep_time=120):

    ranges = np.arange(0, df_missing_stations.shape[0], 
                       batch_size, dtype=int)
    ranges = np.concatenate((ranges, [df_missing_stations.shape[0]]))
    x = 0
    processes = []
    for ii in range(1, len(ranges)):
        print(ranges[ii-1], ranges[ii])
        save_file = f"approximate_stations/{save_file_suffix}_{ii}.csv"
        partitioned_df = df_missing_stations.loc[ranges[ii-1]: 
                                    ranges[ii]].copy().reset_index(drop=True)    
        
        p = Process(target=get_approximate_stations_locations, 
                            args = (partitioned_df, start_stations, save_file))
        #get_approximate_stations_locations(partitioned_df, start_stations, save_file)
        x +=1
        p.start()
        print(x)
        processes.append(p)
        time.sleep(sleep_time)
    for thread in processes:
        thread.join()

# Importing and renaming columns

In [6]:
df_initial = pd.DataFrame()
for file in glob.glob("*.csv"):
    df_initial = df_initial.append(pd.read_csv(file).rename(columns={
        "started_at": "start_time",
        "ended_at": "end_time",
        "start_lat": "start_station_latitude",
        "start_lng": "start_station_longitude",
        "end_lat": "end_station_latitude",
        "end_lng": "end_station_longitude",
        "member_casual": "user_type"
    }))

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (4,5,8,9,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_initial = df_initial.dropna(subset=[
    'start_station_latitude',
    'start_station_longitude',
    'end_station_latitude',
    'end_station_longitude'])

In [8]:
df_initial['start_time'] = pd.to_datetime(df_initial['start_time'])

In [9]:
df_initial = df_initial[df_initial['start_time'] >= datetime.datetime(2021, 1, 1)]

# Finding Standard Stations

In [10]:
start_stations = df_initial[["start_station_name", 
                             'start_station_latitude', 
                             "start_station_longitude"]]\
.sort_values(['start_station_latitude', "start_station_longitude"])\
.drop_duplicates(subset=['start_station_latitude','start_station_longitude'])\
.dropna().drop_duplicates(subset=['start_station_name'])\
.query("start_station_latitude != 0").reset_index(drop=True).rename(
    columns={"start_station_name": "station_name", 
            'start_station_latitude': "station_latitude",
            "start_station_longitude": "station_longitude"})

In [11]:
end_stations = df_initial[["end_station_name", 
                           'end_station_latitude', 
                           "end_station_longitude"]]\
.sort_values(['end_station_latitude', "end_station_longitude"])\
.drop_duplicates(subset=['end_station_latitude','end_station_longitude'])\
.dropna().drop_duplicates(subset=['end_station_name'])\
.query("end_station_latitude != 0").reset_index(drop=True).rename(
    columns={"end_station_name": "station_name", 
            'end_station_latitude': "station_latitude",
            "end_station_longitude": "station_longitude"})

In [12]:
standard_stations = start_stations.append(end_stations).sort_values(
    ['station_latitude', "station_longitude"]).drop_duplicates(subset=['station_name'])

# Trip duration calculation

In [13]:
df_initial['start_time'] = pd.to_datetime(df_initial['start_time'])
df_initial['end_time'] = pd.to_datetime(df_initial['end_time'])
df_initial['duration_sec'] = (df_initial['end_time'] - df_initial['start_time'])/ pd.Timedelta(seconds=1)

df_initial = create_datetime_features(df_initial, "start_time")
df_initial = create_datetime_features(df_initial, "end_time")
df_initial['user_type'] = df_initial['user_type'].replace({"member":"Subscriber", "casual": "Customer"})

Removing 2020 data for irrelevancy 

In [14]:
df_reduced = df_initial.query("start_time_year > 2020").reset_index(drop=True)

In [15]:
del df_initial

# Dropping Trips < X minutes duration, where start = end

X = 4

In [16]:
def flagging_short_trips(x):
    return ((x['end_station_latitude'] == x['start_station_latitude']) &
        (x['end_station_longitude'] == x['start_station_longitude'])) \
            | (x['start_station_name'] == x['end_station_name'])
       

In [ ]:
indexes = df_reduced.apply(flagging_short_trips, axis=1)

In [ ]:
indexes.value_counts()

In [ ]:
df_same_stations = df_reduced.loc[indexes]

In [ ]:
minutes = 4

In [ ]:
df_no_same_stations = df_reduced.loc[~indexes].append(
                        df_same_stations.query(f"duration_sec < {minutes*60}")).reset_index(drop=True)

In [ ]:
df_missing_stations = df_no_same_stations[(df_no_same_stations['start_station_name'].isna()) |\
                                          (df_no_same_stations['end_station_name'].isna())].copy()

df_existing_stations = df_no_same_stations[~((df_no_same_stations['start_station_name'].isna()) |\
                                             (df_no_same_stations['end_station_name'].isna()))].copy()



# Lookup missing Station Names

Approximating missing stations

In [ ]:
df_missing_start_stations_no_duplicates = df_missing_stations.drop_duplicates(
                                        subset=['start_station_latitude', 'start_station_longitude'])
df_missing_start_stations_no_duplicates = df_missing_start_stations_no_duplicates[
                                            df_missing_start_stations_no_duplicates['start_station_name'].isna()]

In [ ]:
df_missing_end_stations_no_duplicates = df_missing_stations.drop_duplicates(
                                        subset=['end_station_latitude', 'end_station_longitude'])
df_missing_end_stations_no_duplicates = df_missing_end_stations_no_duplicates[
                                            df_missing_end_stations_no_duplicates['end_station_name'].isna()]

In [ ]:
approximate_start_stations_df = get_approximate_stations_locations(df_missing_start_stations_no_duplicates, 
                                   start_stations, save_file="approximate_stations/approx_start_stations.csv", 
                                                                   col ="start_station")

In [ ]:
approximate_end_stations_df = get_approximate_stations_locations(df_missing_end_stations_no_duplicates, 
                                   start_stations, save_file="approximate_stations/approx_end_stations.csv", 
                                                                 col ="end_station")

In [ ]:
approximate_start_stations_df = pd.read_csv("approximate_stations/approx_start_stations.csv") 
approximate_end_stations_df = pd.read_csv("approximate_stations/approx_end_stations.csv") 

In [ ]:
df_missing_stations = df_missing_stations.drop(columns=[
    'start_station_approx', 
    'start_station_approx_distance']).merge(
    approximate_start_stations_df[["start_station_approx", 
                             'start_station_latitude',
                             'start_station_longitude',
                             'start_station_approx_distance']],
    on=['start_station_latitude', 
        'start_station_longitude'], how="left")

In [ ]:
df_missing_stations = df_missing_stations.merge(
    approximate_end_stations_df[["end_station_approx", 
                             'end_station_latitude',
                             'end_station_longitude',
                             'end_station_approx_distance']],
    on=['end_station_latitude', 
        'end_station_longitude'], how="left")

In [ ]:
index = df_missing_stations['start_station_approx_distance'] < 500
df_missing_stations.loc[index, 'start_station_name'] = df_missing_stations.loc[index, 'start_station_approx']

index = df_missing_stations['end_station_approx_distance'] < 500
df_missing_stations.loc[index, 'end_station_name'] = df_missing_stations.loc[index, 'end_station_approx']

In [ ]:
df_completed_stations = df_missing_stations.dropna(subset=['start_station_name', 'end_station_name'])

In [ ]:
df_existing_stations = df_existing_stations.drop(columns=['start_station_id', 'end_station_id', 'bike_id', 'rental_access_method', 'Unnamed: 0'])
df_completed_stations = df_completed_stations.drop(columns=['start_station_id', 'end_station_id', 'bike_id', 'rental_access_method', 'Unnamed: 0'])

In [ ]:
df = df_completed_stations.append(df_existing_stations)

In [ ]:
df['end_station_name'].isna().value_counts()

In [ ]:
df.to_csv("baywheels_cleaned.csv")


In [ ]:
standard_stations.to_csv("standard_stations.csv")

In [ ]:
#count = 0
#for idx, i, j in zip(df_missing_stations.index, 
#                     df_missing_stations['start_station_latitude'], 
#                     df_missing_stations['start_station_longitude']):
#    approx_station, approx_distance = calculate_geodesic_distance(start_stations, 
#                        (i, j))
#    df_missing_stations.loc[idx,"start_station_approx"] = approx_station
#    df_missing_stations.loc[idx,"start_station_approx_distance"] = approx_distance
#    count += 1
#    if count%10000 == 0:
#        print(count)
#        df_missing_stations.to_csv("missing_stations.csv")

In [ ]:
#
#import plotly.express as px
#fig = px.density_mapbox(df_initial.head(100000), lat='start_station_latitude', lon='start_station_longitude', radius=2,
#                        center=dict(lat=0, lon=180), zoom=0,
#                        mapbox_style="stamen-terrain")
#fig.update_geos(fitbounds="locations")
#fig.show()

In [ ]:
#from sklearn.metrics.pairwise import haversine_distances
#
#points_in_radians = df_initial[['start_station_latitude','start_station_longitude']].head(10000).apply(np.radians).values
#distances_in_km = haversine_distances(points_in_radians) * 6371

In [ ]:
#distance_matrix = distances_in_km

#clustering = DBSCAN(min_samples=2)
#scaler = MinMaxScaler()
#df = df_initial.copy()
#df[['start_station_latitude', 'start_station_longitude', 
#    'end_station_latitude', 'end_station_longitude']] = \
#                            scaler.fit_transform(df[['start_station_latitude', 'start_station_longitude', 
#                                                     'end_station_latitude', 'end_station_longitude']])
#clusters = clustering.fit_predict(df[['start_station_latitude', 'start_station_longitude']].head(10000))